# FireMon Policy Optimizer Automation

This notebook automates the process of:
1. Opening FireMon Policy Optimizer
2. Processing first two PO tickets
3. Finding policy revision changes
4. Taking screenshots of specific rule changes

## Stage 1: Setup and Configuration
Install and import required libraries

In [ ]:
# Install required packages (optional - only if you want webdriver-manager fallback)
# !pip install selenium webdriver-manager pillow

In [ ]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.edge.service import Service as EdgeService
import time
import os
from datetime import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Configuration - UPDATE THESE VALUES
FIREMON_URL = "https://your-firemon-url.com"  # Replace with your FireMon URL
USERNAME = "your_username"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

# Browser Configuration
BROWSER = "firefox"  # Choose: "firefox" or "edge"
HEADLESS = False  # Set to True for headless mode

# Browser Paths (leave empty to use default system locations)
FIREFOX_BINARY_PATH = ""  # e.g., "/usr/bin/firefox" or "C:\\Program Files\\Mozilla Firefox\\firefox.exe"
FIREFOX_DRIVER_PATH = ""  # e.g., "/usr/local/bin/geckodriver" or "C:\\path\\to\\geckodriver.exe"
EDGE_BINARY_PATH = ""     # e.g., "C:\\Program Files (x86)\\Microsoft\\Edge\\Application\\msedge.exe"

# Common Firefox paths (will try these if FIREFOX_BINARY_PATH is empty)
FIREFOX_COMMON_PATHS = [
    "/usr/bin/firefox",                                    # Linux
    "/Applications/Firefox.app/Contents/MacOS/firefox",   # macOS
    "C:\\Program Files\\Mozilla Firefox\\firefox.exe",    # Windows
    "C:\\Program Files (x86)\\Mozilla Firefox\\firefox.exe"
]

# Common geckodriver paths (will try these if FIREFOX_DRIVER_PATH is empty)
GECKODRIVER_COMMON_PATHS = [
    "/usr/local/bin/geckodriver",          # Linux/macOS
    "/usr/bin/geckodriver",               # Linux
    "geckodriver",                        # Current directory or PATH
    "./geckodriver",                      # Current directory
    "C:\\geckodriver.exe",               # Windows root
    "C:\\webdrivers\\geckodriver.exe"    # Windows webdrivers folder
]

# Create screenshots directory
SCREENSHOT_DIR = "screenshots"
if not os.path.exists(SCREENSHOT_DIR):
    os.makedirs(SCREENSHOT_DIR)
    logger.info(f"Created screenshot directory: {SCREENSHOT_DIR}")

## Stage 2: Initialize WebDriver
Set up Firefox or Edge driver with configurable paths

In [ ]:
def find_executable(paths_list, executable_name):\n    \"\"\"Find executable from a list of common paths\"\"\"\n    for path in paths_list:\n        if os.path.exists(path):\n            logger.info(f\"Found {executable_name} at: {path}\")\n            return path\n    return None\n\ndef initialize_firefox_driver(headless=False):\n    \"\"\"Initialize Firefox WebDriver\"\"\"\n    try:\n        # Setup Firefox options\n        firefox_options = webdriver.FirefoxOptions()\n        firefox_options.add_argument('--start-maximized')\n        \n        if headless:\n            firefox_options.add_argument('--headless')\n        \n        # Find Firefox binary\n        firefox_binary = FIREFOX_BINARY_PATH\n        if not firefox_binary or not os.path.exists(firefox_binary):\n            firefox_binary = find_executable(FIREFOX_COMMON_PATHS, \"Firefox\")\n            if not firefox_binary:\n                logger.warning(\"Firefox binary not found in common locations\")\n            else:\n                firefox_options.binary_location = firefox_binary\n        elif firefox_binary:\n            firefox_options.binary_location = firefox_binary\n        \n        # Find geckodriver\n        geckodriver_path = FIREFOX_DRIVER_PATH\n        if not geckodriver_path or not os.path.exists(geckodriver_path):\n            geckodriver_path = find_executable(GECKODRIVER_COMMON_PATHS, \"geckodriver\")\n        \n        # Initialize Firefox driver\n        if geckodriver_path:\n            service = FirefoxService(executable_path=geckodriver_path)\n            driver = webdriver.Firefox(service=service, options=firefox_options)\n            logger.info(f\"Firefox WebDriver initialized with geckodriver at: {geckodriver_path}\")\n        else:\n            # Try without specifying geckodriver path (assumes it's in PATH)\n            logger.info(\"Trying Firefox WebDriver without specifying geckodriver path...\")\n            driver = webdriver.Firefox(options=firefox_options)\n            logger.info(\"Firefox WebDriver initialized (geckodriver found in PATH)\")\n        \n        return driver\n    \n    except Exception as e:\n        logger.error(f\"Failed to initialize Firefox driver: {str(e)}\")\n        raise e\n\ndef initialize_edge_driver(headless=False):\n    \"\"\"Initialize Edge WebDriver\"\"\"\n    try:\n        # Setup Edge options\n        edge_options = webdriver.EdgeOptions()\n        edge_options.add_argument('--start-maximized')\n        edge_options.add_argument('--disable-blink-features=AutomationControlled')\n        edge_options.add_experimental_option(\"excludeSwitches\", [\"enable-automation\"])\n        edge_options.add_experimental_option('useAutomationExtension', False)\n        \n        if headless:\n            edge_options.add_argument('--headless')\n        \n        # Set Edge binary path if specified\n        if EDGE_BINARY_PATH and os.path.exists(EDGE_BINARY_PATH):\n            edge_options.binary_location = EDGE_BINARY_PATH\n            logger.info(f\"Using Edge binary at: {EDGE_BINARY_PATH}\")\n        \n        # Try with system msedgedriver (no service needed)\n        logger.info(\"Using system Edge driver...\")\n        driver = webdriver.Edge(options=edge_options)\n        logger.info(\"Edge WebDriver initialized with system driver\")\n        \n        return driver\n    \n    except Exception as e:\n        logger.error(f\"Failed to initialize Edge driver: {str(e)}\")\n        raise e\n\ndef initialize_driver(browser_type=\"firefox\", headless=False):\n    \"\"\"Initialize WebDriver (Firefox or Edge)\"\"\"\n    try:\n        if browser_type.lower() == \"firefox\":\n            driver = initialize_firefox_driver(headless)\n        else:  # Edge\n            driver = initialize_edge_driver(headless)\n        \n        # Hide webdriver property (works for both Firefox and Edge)\n        try:\n            driver.execute_script(\"Object.defineProperty(navigator, 'webdriver', {get: () => undefined})\")\n        except:\n            pass  # Some drivers don't support this\n        \n        return driver\n    \n    except Exception as e:\n        logger.error(f\"Failed to initialize {browser_type} driver: {str(e)}\")\n        # Try fallback browser\n        if browser_type.lower() == \"firefox\":\n            logger.info(\"Trying Edge as fallback...\")\n            return initialize_driver(\"edge\", headless)\n        elif browser_type.lower() == \"edge\":\n            logger.info(\"Trying Firefox as fallback...\")\n            return initialize_driver(\"firefox\", headless)\n        else:\n            raise e\n\n# Initialize driver\nlogger.info(f\"Initializing {BROWSER} WebDriver...\")\ndriver = initialize_driver(BROWSER, HEADLESS)\nwait = WebDriverWait(driver, 20)  # 20 second timeout for element waits\n\nlogger.info(f\"WebDriver initialized successfully using {BROWSER}\")"

## Stage 3: Login to FireMon
Navigate to FireMon and perform login

In [ ]:
try:
    # Navigate to FireMon
    logger.info(f"Navigating to FireMon: {FIREMON_URL}")
    driver.get(FIREMON_URL)
    
    # Wait for login page to load
    time.sleep(3)
    
    # Find and fill username field
    username_field = wait.until(EC.presence_of_element_located((By.ID, "username")))
    username_field.clear()
    username_field.send_keys(USERNAME)
    logger.info("Username entered")
    
    # Find and fill password field
    password_field = driver.find_element(By.ID, "password")
    password_field.clear()
    password_field.send_keys(PASSWORD)
    logger.info("Password entered")
    
    # Click login button
    login_button = driver.find_element(By.ID, "login-button")
    login_button.click()
    logger.info("Login button clicked")
    
    # Wait for dashboard to load
    time.sleep(5)
    logger.info("Login successful")
    
except Exception as e:
    logger.error(f"Login failed: {str(e)}")
    raise

## Stage 4: Navigate to Policy Optimizer
Find and click on Policy Optimizer link

In [ ]:
try:
    # Find Policy Optimizer link/menu item
    # Note: Update the selector based on actual FireMon interface
    po_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Policy Optimizer")))
    po_link.click()
    logger.info("Clicked Policy Optimizer link")
    
    # Wait for PO page to load
    time.sleep(5)
    
    # Wait for PO tickets to load
    po_tickets = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "po-ticket")))
    logger.info(f"Found {len(po_tickets)} PO tickets")
    
except Exception as e:
    logger.error(f"Failed to navigate to Policy Optimizer: {str(e)}")
    raise

## Stage 5: Process First Two PO Tickets
Extract rule names and process each ticket

In [ ]:
# Store ticket information
tickets_to_process = []

try:
    # Get first two PO tickets
    po_tickets = driver.find_elements(By.CLASS_NAME, "po-ticket")[:2]
    
    for i, ticket in enumerate(po_tickets):
        # Extract rule name from ticket
        # Note: Update selector based on actual structure
        rule_name = ticket.find_element(By.CLASS_NAME, "rule-name").text
        
        # Find policy revision link
        revision_link = ticket.find_element(By.CLASS_NAME, "policy-revision-link")
        
        tickets_to_process.append({
            'index': i,
            'rule_name': rule_name,
            'revision_link': revision_link
        })
        
        logger.info(f"Ticket {i+1}: Rule name = {rule_name}")
    
    logger.info(f"Extracted info for {len(tickets_to_process)} tickets")
    
except Exception as e:
    logger.error(f"Failed to extract ticket information: {str(e)}")
    raise

## Stage 6: Process Each Ticket
Click revision link, view changes, and capture screenshots

In [ ]:
# Process each ticket
for ticket_info in tickets_to_process:
    try:
        logger.info(f"\nProcessing ticket {ticket_info['index']+1}: {ticket_info['rule_name']}")
        
        # Store current window handle
        main_window = driver.current_window_handle
        
        # Click on policy revision link
        ticket_info['revision_link'].click()
        logger.info("Clicked policy revision link")
        
        # Switch to new window/tab
        time.sleep(2)
        for window_handle in driver.window_handles:
            if window_handle != main_window:
                driver.switch_to.window(window_handle)
                break
        
        # Wait for page to load
        time.sleep(3)
        
        # Find and click "View Changes" toggle button
        view_changes_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'View Changes')]|//button[contains(text(), 'view changes')]|//button[@class='view-changes-toggle']|//button[@id='view-changes']")))
        view_changes_btn.click()
        logger.info("Clicked 'View Changes' button")
        
        # Wait for changes to load
        time.sleep(5)
        
        # Find all rule change rows
        rule_rows = driver.find_elements(By.CLASS_NAME, "rule-change-row")
        logger.info(f"Found {len(rule_rows)} rule change rows")
        
        # Find the specific rule matching our rule name
        found_matching_rule = False
        for row in rule_rows:
            try:
                # Get rule name from row (update selector as needed)
                row_rule_name = row.find_element(By.CLASS_NAME, "rule-name").text
                
                if row_rule_name == ticket_info['rule_name']:
                    logger.info(f"Found matching rule: {row_rule_name}")
                    
                    # Scroll to element
                    driver.execute_script("arguments[0].scrollIntoView(true);", row)
                    time.sleep(1)
                    
                    # Take screenshot
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    screenshot_name = f"{SCREENSHOT_DIR}/PO_ticket_{ticket_info['index']+1}_{ticket_info['rule_name']}_{timestamp}.png"
                    row.screenshot(screenshot_name)
                    logger.info(f"Screenshot saved: {screenshot_name}")
                    
                    found_matching_rule = True
                    break
                    
            except Exception as e:
                logger.warning(f"Error processing row: {str(e)}")
                continue
        
        if not found_matching_rule:
            logger.warning(f"No matching rule found for: {ticket_info['rule_name']}")
            # Take full page screenshot as fallback
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            screenshot_name = f"{SCREENSHOT_DIR}/PO_ticket_{ticket_info['index']+1}_full_page_{timestamp}.png"
            driver.save_screenshot(screenshot_name)
            logger.info(f"Full page screenshot saved: {screenshot_name}")
        
        # Close current tab and switch back to main window
        driver.close()
        driver.switch_to.window(main_window)
        time.sleep(2)
        
    except Exception as e:
        logger.error(f"Error processing ticket {ticket_info['index']+1}: {str(e)}")
        # Try to switch back to main window
        try:
            driver.switch_to.window(main_window)
        except:
            pass

## Stage 7: Cleanup
Close browser and summarize results

In [ ]:
# Print summary
logger.info("\n=== AUTOMATION COMPLETE ===")
logger.info(f"Screenshots saved in: {SCREENSHOT_DIR}")

# List all screenshots
screenshots = [f for f in os.listdir(SCREENSHOT_DIR) if f.endswith('.png')]
logger.info(f"Total screenshots captured: {len(screenshots)}")
for screenshot in screenshots:
    logger.info(f"  - {screenshot}")

In [ ]:
# Close browser
driver.quit()
logger.info("Browser closed")

## Troubleshooting Tips

1. **WebDriver Setup:**
   - **Firefox (Default)**: Download geckodriver from https://github.com/mozilla/geckodriver/releases
   - **Edge (Fallback)**: Uses system-installed driver (Windows 10/11)
   - Place geckodriver in one of the common paths or set `FIREFOX_DRIVER_PATH`
   - Set browser binary paths if installed in non-standard locations

2. **Firefox Configuration:**
   - Set `FIREFOX_BINARY_PATH` if Firefox is installed in a custom location
   - Set `FIREFOX_DRIVER_PATH` to point to your geckodriver executable
   - Common geckodriver locations: `/usr/local/bin/`, `./`, current PATH

3. **Edge Configuration:**
   - Set `EDGE_BINARY_PATH` if Edge is in a non-standard location
   - Edge driver (msedgedriver) is included with Edge installation
   - No additional driver download needed for Edge

4. **Browser Selection:**
   - Set `BROWSER = \"firefox\"` (default, more stable for automation)
   - Set `BROWSER = \"edge\"` (fallback, Windows only)
   - Automatic fallback if primary browser fails

5. **Login Issues:**
   - Check if username/password field IDs are correct
   - Verify credentials are correct
   - Check if there's a captcha or 2FA

6. **Element Not Found:**
   - Use browser developer tools to inspect elements
   - Update selectors (ID, class name, XPath) accordingly
   - Increase wait times if elements load slowly

7. **Screenshot Issues:**
   - Ensure screenshot directory has write permissions
   - Check if element is visible before taking screenshot
   - Use full page screenshot as fallback

8. **Common Selectors to Update:**
   - Login: `#username`, `#password`, `#login-button`
   - PO Tickets: `.po-ticket`, `.rule-name`, `.policy-revision-link`
   - Changes: `.view-changes-toggle`, `.rule-change-row`

9. **Quick Setup Guide:**
   ```bash
   # Download geckodriver
   wget https://github.com/mozilla/geckodriver/releases/latest/download/geckodriver-v0.33.0-linux64.tar.gz
   tar -xzf geckodriver-v0.33.0-linux64.tar.gz
   sudo mv geckodriver /usr/local/bin/
   chmod +x /usr/local/bin/geckodriver
   ```"